In [ ]:
column_headers = [
    "Unique identifier",
    "Last name / organization",
    "First name",
    "Middle initial",
    "Credential ('', 'M.D.', 'D.O.', 'MD','DO', 'OTR', 'D.C.', ...)",
    "Gender",
    "Entity ('I' for individual, 'O' for organization)",
    "Address line 1",
    "Address line 2",
    "City",
    "Zip",
    "State",
    "Country",
    "Specialty ('Internal Medicine', 'Pathology', ...)",
    "Participates in medicare ('Y' or 'N')",
    "Place of service ('F' for facility or 'O' for other)",
    "HCPCS code (a procedure code)",
    "HCPCS code description",
    "HCPCS drug indicator ('Y' or 'N')",
    "Line service count (people, hours, miles, ...)",
    "Beneficiary unique count (number of distinct beneficiary, possibly receiving many procedures)",
    "Beneficiary day service count (number of distinct procedures, possibly on the same person)",
    "Average medicare allowed amount",
    "Standard deviation",
    "Average submitted charges amount",
    "Standard deviation",
    "Average medicare payment amount",
    "Standard deviation"
]
for i in range(0, len(column_headers)):
    print str(i) + ": " + column_headers[i]

In [6]:
# Computing statistics, building dictionaries:

from math import floor

procedure_dictionary = dict()
procedure_count = dict()
procedure_total_submitted = dict()
procedure_total_allowed = dict()

infile = open("provider_utilization_2013.txt","r")
line = infile.readline()
line = infile.readline()
#for c in range(10000):
while True:   
    line = infile.readline()
    if(line == ""):
        break
    tokens = line.split("\t")
    
    procedure_id = tokens[16]
    
    procedure_dictionary[procedure_id] = tokens[17]
    
    number_of_procedures = int(tokens[21])
    avg_allowed_charge = float(tokens[22])
    avg_submitted_charge = float(tokens[24])
       
    if procedure_id not in procedure_count:
        procedure_count[procedure_id] = number_of_procedures
        procedure_total_submitted[procedure_id] = avg_submitted_charge * number_of_procedures
        procedure_total_allowed[procedure_id] = avg_allowed_charge * number_of_procedures
    else:
        procedure_count[procedure_id] += number_of_procedures
        procedure_total_submitted[procedure_id] += avg_submitted_charge * number_of_procedures
        procedure_total_allowed[procedure_id] += avg_allowed_charge * number_of_procedures
infile.close()

# procedure_avg_allowed = dict()
# procedure_avg_submitted = dict()
# for procedure_id in procedure_count:
#     procedure_avg_submitted[procedure_id] = procedure_total_submitted[procedure_id] / procedure_count[procedure_id]
#     procedure_avg_allowed[procedure_id] = procedure_total_allowed[procedure_id] / procedure_count[procedure_id]

    
# procedure_distribution = dict()
# infile = open("provider_utilization_2013.txt","r")
# line = infile.readline()
# line = infile.readline()
# #for c in range(10000):
# while True:
#     line = infile.readline()
#     if(line == ""):
#         break
#     tokens = line.split("\t")
    
#     procedure_id = tokens[16]
#     number_of_procedures = int(tokens[21])
#     avg_submitted_charge = float(tokens[24])
#     state = tokens[11]
    
#     bin_size = procedure_avg_allowed[procedure_id]
    
#     if (procedure_id, state) not in procedure_distribution:
#         procedure_distribution[(procedure_id, state)] = [0 for i in range(8)]
        
#     i = int(floor(avg_submitted_charge / bin_size))
#     if i > 7:
#         i = 7
#     procedure_distribution[(procedure_id, state)][i] += number_of_procedures
    
# infile.close()

In [9]:
# Load procedure-related mySQL tables 

year = 2013

import os, django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import ProcedureDescriptor, ProcedureAvgCharges, ProcedureCounts

for code in procedure_dictionary:
    if(ProcedureDescriptor.objects.filter(code = code).count() == 0):
        descriptor = ProcedureDescriptor(code = code, descriptor = procedure_dictionary[code])
        descriptor.save()    

# for code in procedure_dictionary:
#     descriptor = ProcedureDescriptor.objects.get(code = code)
#     charges = ProcedureAvgCharges(
#         descriptor = descriptor,
#         year = year,
#         allowed = procedure_avg_allowed[code],
#         submitted = procedure_avg_submitted[code])
#     charges.save()
    

#for pair in procedure_distribution:
#    (code, state) = pair
#    descriptor = ProcedureDescriptor.objects.get(code = code)
#    i = 0
#    for value in procedure_distribution[pair]:
#        counts = ProcedureCounts(
#            descriptor = descriptor, 
#            year = year,
#            state = state,
#            index = i,
#            value = value)
#        counts.save()
#        i += 1

In [11]:
year = 2013
reset = True

import django
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "medissect.settings")
django.setup()
from explorer.models import Procedure, ProcedureDescriptor, Provider

if reset:
  Procedure.objects.all().delete()
  
infile = open("provider_utilization_2013.txt","r")
line = infile.readline()
line = infile.readline()
# for c in range(10000):
while True:
    line = infile.readline()
    if(line == ""):
        break
    tokens = line.split("\t")
    
    procedure_id = tokens[16]
    provider_id = int(tokens[0])
    procedure_count = int(tokens[21])
    avg_submitted_charge = float(tokens[24])
    state = tokens[11]
    
    try:
        descriptor = ProcedureDescriptor.objects.get(code = procedure_id)
    except ProcedureDescriptor.DoesNotExist:
        continue
        
    try:
        provider = Provider.objects.get(npi = provider_id)
    except Provider.DoesNotExist:
        continue
    
    procedure = Procedure(
        descriptor = descriptor,
        provider = provider,
        year = year,
        procedure_count = int(tokens[21]),
        beneficiary_count = int(tokens[20]),
        line_service_count = float(tokens[19]),
        allowed_avg = float(tokens[22]),
        allowed_std = float(tokens[23]),
        submitted_avg = float(tokens[22]),
        submitted_std = float(tokens[23]),
        payed_avg = float(tokens[22]),
        payed_std = float(tokens[23]),
    )
    procedure.save()
infile.close()